# VDCNN Implementation In Torch  

In [1]:
# importing required libraries
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import datasets
from torchtext import data
import nltk
import pyprind
import math

In [2]:
# s = r'''abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’"/|$%ˆ&*˜‘+=<>()[]{}'''
# nltk.regexp_tokenize("I love,  this ship", s)

In [3]:
def tokenize(text):
    s = 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’"/|$%ˆ&*˜‘+=<>()[]{} '
    return [l for l in list(text.lower()) if l in s]

In [4]:
TEXT = data.Field(tokenize=tokenize, fix_length=1014)
LABEL = data.LabelField(dtype=torch.float)

In [5]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [7]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [8]:
# TEXT.vocab.stoi
LABEL.vocab.freqs

Counter({'pos': 12500, 'neg': 12500})

In [9]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device
)

cuda


In [10]:
for batch in train_iterator:
    te, y = batch.text, batch.label
    break

/home/neo/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:117: UserWarning: 
    Found GPU0 GeForce GT 755M which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [11]:
te.shape[0]

1014

In [ ]:
class ConvolutionalBlockRes(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=1, shortcut=False, pool_type="max_pool"):
        super().__init__()
        self.shortcut = shortcut
        self.pool_type = pool_type
        self.conv_1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=1)
        self.batch_norm_1 = nn.BatchNorm1d(out_channels)
        self.conv_2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding=1)
        self.batch_norm_2 = nn.BatchNorm1d(out_channels)

        if shortcut is True:
            self.conv_res = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=2)
            self.batch_norm_res = nn.BatchNorm1d(out_channels)
    
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(self.batch_norm_1(out))

        out = self.conv_2(out)
        out = F.relu(self.batch_norm_2(out))
        
        # downsampled
        if self.pool_type == "k_max":
            k_ = math.ceil(out.shape[2]/2.0)
            out = downsample_k_max_pool(out, k=k_, dim=2)[0]
        else:
            out = downsample_max_pool(out, 3, 2)

        if self.shortcut is True:
            residual = self.conv_res(x)
            residual = F.relu(self.batch_norm_res(residual))
            out = out + residual
        return out

class ConvolutionalIdentityBlock(nn.Module):
    def __init__(self, in_channels, kernel_size, padding=1, shortcut=False):
        super().__init__()
        
        self.shortcut = shortcut
        self.conv_1 = nn.Conv1d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, padding=1)
        self.batch_norm_1 = nn.BatchNorm1d(in_channels)
        self.conv_2 = nn.Conv1d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, padding=1)
        self.batch_norm_2 = nn.BatchNorm1d(in_channels)
    
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(self.batch_norm_1(out))

        out = self.conv_2(out)
        out = F.relu(self.batch_norm_2(out))

        if self.shortcut is True:
            out = out + x
        else:
            out = out

        return out

def downsample_max_pool(x, kernel_size, stride):
    pool = nn.MaxPool1d(kernel_size=kernel_size, stride=stride, padding=1)
    return pool(x)


def downsample_k_max_pool(inp, k, dim):
    return inp.topk(k, dim)

In [ ]:
class VDCNN(nn.Module):
    def __init__(self, embedding_dim, vocab_size, n_classes):
        super().__init__()
        self.embedding = nn.Embedding(embedding_dim=embedding_dim, num_embeddings=vocab_size)
        
        self.conv_64 = nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=3, padding=1)
        
        self.id_64 = ConvolutionalIdentityBlock(64, kernel_size=3, padding=1, shortcut=True)
        
        self.res_128 = ConvolutionalBlockRes(in_channels=64, out_channels=128, kernel_size=3, padding=1, shortcut=True, pool_type="k_max")
        
        self.id_128 = ConvolutionalIdentityBlock(128, kernel_size=3, padding=1, shortcut=True)
        
        self.res_256 = ConvolutionalBlockRes(in_channels=128, out_channels=256, kernel_size=3, padding=1, shortcut=True, pool_type="k_max")

        self.id_256 = ConvolutionalIdentityBlock(256, kernel_size=3, padding=1, shortcut=True)
        
        self.res_512 = ConvolutionalBlockRes(in_channels=256, out_channels=512, kernel_size=3, padding=1, shortcut=True, pool_type="k_max")
        
        self.id_512 = ConvolutionalIdentityBlock(512, kernel_size=3, padding=1, shortcut=True)
        
        self.linear_1 = nn.Linear(8*512, 2048)
        self.linear_2 = nn.Linear(2048, 2048)
        self.linear_3 = nn.Linear(2048, n_classes)
        
    def forward(self, inp):
        
        # [sent_length, batch_size]
        inp = inp.permute(1, 0)
        
        # [batch_size, sent_length]
        embedded = self.embedding(inp)
        
        # [batch_size, sent_lenght, emb_dim]
        embedded = embedded.permute(0, 2, 1)
        
        # [batch_size, emb_dim, sent_length]
        out = self.conv_64(embedded)
#         print(out.shape)
        
        # [batch_size, 64, sent_length]
        out = self.id_64(out)
#         print(out.shape)
        
#         # [batch_size, 64, sent_length]
        out = self.res_128(out)
#         print(out.shape)
        
#         # [batch_size, 128, sent_length/2]
        out = self.id_128(out)
#         print(out.shape)
        
#         # [batch_size, 128, sent_length/2]
        out = self.res_256(out)
#         print(out.shape)
        
#         # [batch_size, 256, sent_length/4]
        out = self.id_256(out)
#         print(out.shape)
        
#         # [batch_size, 256, sent_length/4]
        out = self.res_512(out)
#         print(out.shape)
        
#         # [batch_size, 512, sent_length/8]
        out = self.id_512(out)
#         print(out.shape)
        
#         # [batch_size, 512, sent_length/8]
        out = downsample_k_max_pool(out, k=8, dim=2)[0]
#         return k_max_pooled
#         print(out.shape)
        
        out = out.reshape(out.shape[0], -1)
#         print(out.shape)
        # [batch_size, 512, 8]
        out = self.linear_1(out)
        
        # [batch_size, 4096]
        out = self.linear_2(out)
#         print(out.shape)

        # [batch_size, 512, 2048
        out = self.linear_3(out)
#         print(out.shape)
        
        # [batch_size, n_class]
        
        return out

In [20]:
a = model(te)
# a.shape

# a = a.squeeze(1)
# a

# a.shape

# y

# preds = torch.round(torch.sigmoid(a))
# preds
# #     correct = (preds == y).float()
# #     acc = correct.sum()/float(len(correct))

# correct = (preds == y).float()
# correct.sum()/len(correct)

# binary_accuracy(a, y)

NameError: name 'model' is not defined

In [ ]:
embedding_dim = 16
vocab_size = len(TEXT.vocab.stoi)
n_classes = 1

model = VDCNN(embedding_dim, vocab_size, n_classes)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

In [ ]:
print(model)

VDCNN(
  (embedding): Embedding(65, 16)
  (conv_64): Conv1d(16, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (id_64): ConvolutionalIdentityBlock(
    (conv_1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (batch_norm_1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (batch_norm_2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_128): ConvolutionalBlockRes(
    (conv_1): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (batch_norm_1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (batch_norm_2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_res): Conv1d(64, 128, kernel_size=(1,), stride=(2,))
    (batch_norm_res): BatchNorm1d(128, eps=1

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    preds = torch.round(torch.sigmoid(preds))
    correct = (preds == y).float()
    acc = correct.sum()/float(len(correct))
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    bar = pyprind.ProgBar(len(iterator), bar_char='█')
#     bar = pyprind.ProgBar(100, bar_char='█')

    for i, batch in enumerate(iterator):
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
#         print(predictions.shape, batch.Label.shape, model(batch.Text).shape)
        loss = criterion(predictions, batch.label)
#         print(loss.shape)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        bar.update()
        
#         if i > 100:
#             break
#         break
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        bar = pyprind.ProgBar(len(iterator), bar_char='█')
#         bar = pyprind.ProgBar(100, bar_char='█')
        for i, batch in enumerate(iterator):

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            bar.update()
#             if i > 100:
#                 break
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# te.shape

# out = model(te).squeeze(1)
# out.shape

# loss = criterion(out, y)
# loss.shape

# loss

# sig = torch.sigmoid(out)
# sig

# preds = torch.round(sig)
# preds.shape

# preds

# correct = (preds == y).float()
# correct

In [ ]:
N_EPOCHS = 50

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:33
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 01 | Train Loss: 0.745 | Train Acc: 50.25% | Val. Loss: 0.692 | Val. Acc: 51.19% |



Total time elapsed: 00:06:31
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:27
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 02 | Train Loss: 0.592 | Train Acc: 65.65% | Val. Loss: 0.679 | Val. Acc: 69.44% |



Total time elapsed: 00:06:33
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:28
0% [██████████████████████████████] 100% | ETA: 00:00:00

...


Total time elapsed: 00:06:38
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:31
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 20 | Train Loss: 0.151 | Train Acc: 94.02% | Val. Loss: 0.541 | Val. Acc: 80.87% |



Total time elapsed: 00:06:35
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:31
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 21 | Train Loss: 0.139 | Train Acc: 94.64% | Val. Loss: 0.579 | Val. Acc: 83.00% |



Total time elapsed: 00:06:37
0% [██                            ] 100% | ETA: 00:12:41

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

In [78]:
def predict_sentiment(sentence):
    tokenized = tokenize(sentence)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    
    tensor = tensor.unsqueeze(1)
#     print(tensor.shape)
    prediction = model(tensor).squeeze(1)
#     print(prediction)
    preds, ind= torch.round(torch.sigmoid(tensor))
#     print(preds)
    return preds, ind

In [79]:
# text = "My voice range is A2-C5. My chest voice goes up to F4. Included sample in my higher chest range. What is my voice type?"
# predict_sentiment(text)